<a href="https://colab.research.google.com/github/elangbijak4/Security_Research/blob/main/Tool2_Almudaya_File_Analysis_use_CVE_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import requests
import json
import os
from datetime import datetime

In [3]:
def generate_file_profile(file_path):
    file_profile = {}
    file_profile['file_name'] = os.path.basename(file_path)
    file_profile['file_size'] = os.path.getsize(file_path)
    file_profile['file_extension'] = os.path.splitext(file_path)[1]
    file_profile['creation_date'] = datetime.fromtimestamp(os.path.getctime(file_path)).strftime('%Y')
    return file_profile

def get_cve_by_attributes(attributes):
    cve_list = []
    url = "https://cve.circl.lu/api/search/{keyword}"
    keywords = [attributes['file_extension'][1:], attributes['creation_date']]

    for keyword in keywords:
        response = requests.get(url.format(keyword=keyword))
        if response.status_code == 200:
            cve_data = response.json()
            cve_list.extend(cve_data)
        else:
            print(f"Gagal mengambil data CVE untuk {keyword}. Status kode: {response.status_code}")

    return cve_list

def analyze_file_against_cve(file_profile, cve_list):
    relevant_cves = []
    for cve in cve_list:
        if file_profile['file_extension'][1:] in cve['summary'] and file_profile['creation_date'] in cve['Published']:
            relevant_cves.append(cve)

    return relevant_cves

def display_cve_analysis(relevant_cves):
    if not relevant_cves:
        print("Tidak ditemukan CVE yang relevan dengan file ini.")
    else:
        for cve in relevant_cves:
            print(f"\nID CVE: {cve.get('id')}")
            print(f"Deskripsi: {cve.get('summary')}")
            print(f"Tanggal Diumumkan: {cve.get('Published')}")
            print(f"Skor CVSS: {cve.get('cvss')}")
            print(f"Kritisitas: {cve.get('cvss-vector')}")
            print("\nReferensi:")
            for reference in cve.get('references', []):
                print(f"- {reference}")

# Contoh penggunaan
file_path = "/content/pariwisata 39.csv"  # Ganti dengan path file yang diupload
file_profile = generate_file_profile(file_path)
print(file_profile)

cve_list = get_cve_by_attributes(file_profile)
relevant_cves = analyze_file_against_cve(file_profile, cve_list)
display_cve_analysis(relevant_cves)

{'file_name': 'pariwisata 39.csv', 'file_size': 15256, 'file_extension': '.csv', 'creation_date': '2024'}
Gagal mengambil data CVE untuk csv. Status kode: 404
Gagal mengambil data CVE untuk 2024. Status kode: 404
Tidak ditemukan CVE yang relevan dengan file ini.
